In [1]:
import os
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

# Carrega o CSV do diretório data
csv_path = os.path.join('..', 'data', 'TweetsWithTheme.csv')
df = pd.read_csv(csv_path)

print(f"Dataset carregado com {len(df)} linhas")
print(f"Colunas disponíveis: {df.columns.tolist()}")

# Função para limpar texto
def clean_text(text):
    if not isinstance(text, str):
        return ''
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)
    # Remove mentions (@usuario)
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags (#tema)
    text = re.sub(r'#\w+', '', text)
    # Remove caracteres especiais e números, mantendo apenas letras e acentos
    text = re.sub(r'[^a-záéíóúàâêôãõç\s]', '', text, flags=re.IGNORECASE)
    # Remove espaços múltiplos
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Encontra coluna de texto
text_col = next((col for col in df.columns if 'text' in col.lower() or 'tweet' in col.lower()), None)

if text_col:
    print(f"\nLimpando coluna: {text_col}")
    df['cleaned_text'] = df[text_col].apply(clean_text)
    print("Limpeza concluída!")
else:
    print("Coluna de texto não encontrada!")

df.head()

Dataset carregado com 61591 linhas
Colunas disponíveis: ['id', 'tweet_text', 'tweet_date', 'sentiment', 'query_used']

Limpando coluna: tweet_text
Limpeza concluída!


,id,tweet_text,tweet_date,sentiment,query_used,cleaned_text
0,1026606627041026050,ela vendo a família :( FUGIDINHA VERA LÚCIA,Mon Aug 06 23:11:07 +0000 2018,Negativo,vera lucia :(,ela vendo a família FUGIDINHA VERA LÚCIA
1,1026266117646561281,Eu odeio o fato que muito provavelmente meu vo...,Mon Aug 06 00:38:04 +0000 2018,Negativo,ciro :(,Eu odeio o fato que muito provavelmente meu vo...
2,1025889796097564672,"Ciro ""mita"" tanto quanto aquele outro, não que...",Sat Aug 04 23:42:42 +0000 2018,Negativo,ciro :(,Ciro mita tanto quanto aquele outro não quero ...
3,1026960047937646598,Raiva do Bolsonaro agora :),Tue Aug 07 22:35:30 +0000 2018,Positivo,bolsonaro :),Raiva do Bolsonaro agora
4,1026447105932623872,Reflexos da alienação petista comandada por Lu...,Mon Aug 06 12:37:15 +0000 2018,Negativo,lula :(,Reflexos da alienação petista comandada por Lu...


In [2]:
from transformers import pipeline
import numpy as np

# Carrega o modelo BERT para análise de sentimentos em português
print("Carregando modelo BERT para análise de sentimentos...")
sentiment_pipeline = pipeline("sentiment-analysis", 
                             model="neuralmind/bert-base-portuguese-cased")

print("Modelo carregado! Analisando sentimentos em batches...\n")

# Função para analisar sentimentos em batches (muito mais rápido!)
def analyze_sentiments_batch(texts, batch_size=32):
    sentiments = []
    confidences = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        
        # Remove textos vazios
        batch = [text if text and len(text.strip()) > 0 else "neutro" for text in batch]
        
        try:
            results = sentiment_pipeline(batch, truncation=True, max_length=512)
            for result in results:
                sentiments.append(result['label'])
                confidences.append(result['score'])
        except Exception as e:
            print(f"Erro no batch {i}: {e}")
            sentiments.extend(['NEUTRAL'] * len(batch))
            confidences.extend([0.0] * len(batch))
        
        # Mostra progresso
        if (i + batch_size) % (batch_size * 10) == 0:
            print(f"Processadas {min(i + batch_size, len(texts))}/{len(texts)} linhas")
    
    return sentiments, confidences

# Analisa sentimentos
texts = df['cleaned_text'].fillna('').tolist()
sentiments, confidences = analyze_sentiments_batch(texts, batch_size=32)

df['sentiment_pred'] = sentiments
df['confidence'] = confidences

print("\n✅ Análise de sentimentos concluída!\n")
print("Distribuição de sentimentos:")
print(df['sentiment_pred'].value_counts())
print(f"\nConfiança média: {df['confidence'].mean():.3f}")

df[['cleaned_text', 'sentiment_pred', 'confidence']].head(10)

Carregando modelo BERT para análise de sentimentos...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


Modelo carregado! Analisando sentimentos em batches...

Processadas 320/61591 linhas
Processadas 640/61591 linhas
Processadas 960/61591 linhas
Processadas 1280/61591 linhas
Processadas 1600/61591 linhas
Processadas 1920/61591 linhas
Processadas 2240/61591 linhas
Processadas 2560/61591 linhas
Processadas 2880/61591 linhas
Processadas 3200/61591 linhas
Processadas 3520/61591 linhas
Processadas 3840/61591 linhas
Processadas 4160/61591 linhas
Processadas 4480/61591 linhas


KeyboardInterrupt: 